In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import contextily as cx
import plotly.graph_objects as go
import geopandas as gpd
import os
import matplotlib
import subprocess
import torch
import matplotlib.pyplot as plt
import pygsp
import warnings, re

from shapely.geometry import MultiPoint
from sklearn.cluster import KMeans
from tsmoothie import LowessSmoother, ExponentialSmoother
from pyprojroot import here
from scipy.spatial import ConvexHull
from torch_geometric.utils import dense_to_sparse

import source.nn.models as models
import source.utils.utils as utils
import source.utils.fault_detection as fd

from source.utils.utils import roc_params, compute_auc, get_auc, best_mcc, best_f1score, otsuThresholding
from source.utils.utils import synthetic_timeseries
from source.utils.utils import plotly_signal

from importlib import reload
models = reload(models)
utils = reload(utils)
fd = reload(fd)

from pyprojroot import here
root_dir = str(here())

insar_dir = os.path.expanduser('~/data/raw/')
data_path = root_dir + '/data/interim/'

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'DejaVu Serif'})

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

def interpolate_displacement(df):
    interpolated_df = df.set_index('timestamp').resample('6D').ffill()
    interpolated_df['displacement'] = (
                                       df[['timestamp','displacement']].set_index('timestamp')
                                                                       .resample('6D')
                                                                       .interpolate(method='linear')
                                      )
    return interpolated_df

def smoothing(frac):
    def smoothing_(x):
        lowess_smoother = LowessSmoother(smooth_fraction=frac, iterations=1) #0.075 
        lowess_smoother.smooth(x)
        return lowess_smoother.smooth_data[0]
    return smoothing_


def preprocess(df_orig):
    df = df_orig.copy()
    df = (df.groupby('pid', as_index=False)
                    .apply(interpolate_displacement)
                    .reset_index().drop('level_0', axis=1)
                    )

    df['smoothed'] = df.groupby('pid',as_index=False).displacement.transform(smoothing(50/df.timestamp.nunique()))
    return df


def resample_df(df):
    df_resampled = df.set_index('timestamp').resample('6D').ffill()
    return df_resampled

def interpolate_signals(df_orig):
    df = df_orig.set_index('timestamp').resample('6D').ffill()
    df[['displacement','signal']] = ( df_orig[['timestamp','displacement', 'signal']].set_index('timestamp')
                                      .resample('6D')
                                      .interpolate(method='linear')
    )

    # Apply smoothing to 'displacement'
    smooth_func = smoothing(frac=50 / df.index.nunique())
    df['smoothed'] = smooth_func(df['signal'].values)

    return df


def evo(x):
    return np.abs(x.values[-1] - x.values[0])

def grad(x):
    return np.max(np.gradient(x.values))


# Function to calculate the area of the convex hull
def convex_hull_area(df):
    points = df[['easting', 'northing']].values
    hull = ConvexHull(points)
    return hull.volume


### FORMATTING DATA

Requires insar datasets to be saved as csv files

In [ ]:
# OSLO
df_orig = pd.read_csv(insar_dir + "2022/L2B_066_0717_IW3_VV.csv")
lat_min, lat_max, lon_min, lon_max = (59.91, 59.925, 10.69, 10.73)

# SELECT AND FORMAT DATA
df = df_orig.copy()

df = df[ (df.longitude>lon_min) & (df.longitude<=lon_max) &
            (df.latitude>lat_min) & (df.latitude<=lat_max)  ]

# Selection relevant columns
date_cols = sorted([col for col in df.columns if "20" in col]) #columns named after timestamps
keep_cols = date_cols #list with variables to keep from dataframe
id_cols = ['pid', 'latitude', 'longitude', 'easting', 'northing', 'mean_velocity']
keep_cols.extend(id_cols)
df = df[keep_cols]  #replacing old df for memory efficiency

# Formatting from wide to tall dataframe
# Uses a single column for timestamp and a column for displacement
# Number of rows = number of pixels * number of timestamps
df = df.melt(id_vars=id_cols, value_vars=date_cols,
                var_name='timestamp', value_name='displacement').sort_values('pid')
df.timestamp = pd.to_datetime(df.timestamp)

df.sort_values(['pid','timestamp'], inplace=True)

df.to_parquet('/home/vitorro/Repositories/stae/data/interim/df_Oslo.parq')
print(f'{df.pid.nunique()} pids')


In [ ]:
# MALMO
df_orig = pd.read_csv(insar_dir + "2021/066_0742_iw1_vv.csv")
# lat_min, lat_max, lon_min, lon_max = (55.575, 55.60, 12.9,13.0)
# lat_min, lat_max, lon_min, lon_max = (55.55, 55.575, 12.9,13.0)
lat_min, lat_max, lon_min, lon_max = (55.55, 55.60, 12.9,13.0)

# SELECT AND FORMAT DATA
df = df_orig.copy()

df = df[ (df.longitude>lon_min) & (df.longitude<=lon_max) &
            (df.latitude>lat_min) & (df.latitude<=lat_max)  ]

# Selection relevant columns
date_cols = sorted([col for col in df.columns if "20" in col]) #columns named after timestamps
keep_cols = date_cols #list with variables to keep from dataframe
id_cols = ['pid', 'latitude', 'longitude', 'easting', 'northing', 'mean_velocity']
keep_cols.extend(id_cols)
df = df[keep_cols]  #replacing old df for memory efficiency

# Formatting from wide to tall dataframe
# Uses a single column for timestamp and a column for displacement
# Number of rows = number of pixels * number of timestamps
df = df.melt(id_vars=id_cols, value_vars=date_cols,
                var_name='timestamp', value_name='displacement').sort_values('pid')
df.timestamp = pd.to_datetime(df.timestamp)

# OLD DATASET ONLY HAS GOOD DATA AFTER JUNE 2016
df = df[df.timestamp>='2016-06-01'].copy()
df.reset_index(drop=True, inplace=True)
df.sort_values(['pid','timestamp'], inplace=True)

df.to_parquet('/home/vitorro/Repositories/stae/data/interim/df_Malmo.parq')
print(f'{df.pid.nunique()} pids')


### SELECTING CLEAN PARTS OF THE DATA

Preprocesses the data to interpolate and denoise  
Extracts some simple features  
Saves the dataset  
Runs app for dataset selection

In [ ]:
df = preprocess(df)

In [ ]:
df_test = (
            df.drop_duplicates('pid')[['pid','latitude','longitude','easting','northing']]
            .merge(df.groupby('pid', as_index=False).smoothed.agg([np.ptp, evo, grad]), on='pid', how='left')
)

In [ ]:
data_path = '/home/vitorro/Repositories/stae/data/interim/df_Oslo_features.parq'
df_test.to_parquet(data_path)

In [ ]:
# THE APP ALLOWS THE SELECTION OF SUBDATASETS BASED ON THE FEATURES
data_path = '/home/vitorro/Repositories/stae/data/interim/df_Oslo_features.parq'
app_process = subprocess.run(['python', '../source/utils/dash_select.py', data_path])

In [ ]:
app_process.kill()

### GENERATE ANOMALOUS DATASETS

#### Geological anomaly (affected area)

In [ ]:
basedata_path = root_dir + "/data/interim/TGRS_basedata/"
dataset_path = root_dir + "/data/datasets/Geological_anomaly/"

In [ ]:
# Generates dataframes with anoamlous data
warnings.simplefilter("ignore", DeprecationWarning)

random_seed = 0
np.random.seed(random_seed) # 42 for training, 0 for testing

onset_options = [1, 2, 3]
transient_options = [1, 2, 3]
deformation_options = [1, 2]
num_samples = 2
anomaly_radius = 40
mask_size = 250
anomaly_level_factor = 0.75

selected_area_filenames = []
df_orig_names = []
for filename in os.listdir(basedata_path):
    if 'area' in filename and filename.endswith('.parq'):
        selected_area_filenames.append(filename)
        df_orig_names.append(filename.split('.')[0][:-6])

df_orig_names = list(set(df_orig_names))
df_orig_dict = {name:pd.read_parquet(basedata_path + name + '.parq') for name in df_orig_names}

metadata = {
    'seed': random_seed,
    'onset_options': onset_options,
    'transient_options': transient_options,
    'deformation_options': deformation_options,
    'num_samples': num_samples,
    'anomaly_radius': anomaly_radius,
    'mask_size': mask_size,
    'anomaly_level_factor': anomaly_level_factor,
    'basedata_path': basedata_path,
    'dataset_path': dataset_path,
    'selected_area_filenames': selected_area_filenames,
    'df_orig_names': df_orig_names,
}
torch.save(metadata, dataset_path+"data_metadata.pt")

for selected_area_filename in selected_area_filenames:

    area_name = selected_area_filename.split('.')[0]
    
    df_selected = pd.read_parquet(basedata_path + selected_area_filename)

    mask = ( (df_selected.easting < (df_selected.easting.min()+mask_size))
              & (df_selected.northing < (df_selected.northing.min()+mask_size))
    )
    masked_pids = df_selected[mask].pid.unique()
  
    df_orig_name = area_name[:-6]
    df_orig = df_orig_dict[df_orig_name][df_orig_dict[df_orig_name].pid.isin(masked_pids)].copy()

    df_area = []
    for onset in onset_options:
        for transient in transient_options:
            for deformation in deformation_options:
                for seed in range(num_samples):

                    df = df_orig.copy()
                    df = df.groupby('pid', as_index=False).apply(resample_df).reset_index().drop('level_0', axis=1)

                    node_positions = df.drop_duplicates('pid')[['easting','northing']].values
                    anomaly_matrix, label = utils.add_anomaly(node_positions, df.timestamp.nunique(),
                                                              anomaly_radius,
                                                              onset,
                                                              transient,
                                                              deformation)
                    anomaly_level = df_orig.groupby('pid').displacement.agg(np.ptp).quantile(anomaly_level_factor)
                    df['signal'] = df.displacement + anomaly_level*anomaly_matrix.reshape((-1,))
                    df['label'] = label.reshape((-1,))

                    df_stamps = df[df.timestamp.isin(df_orig.timestamp.unique())]
                    df[['displacement','signal','smoothed']] =  (df_stamps.groupby('pid', as_index=False)
                                                                 .apply(interpolate_signals, include_groups=True)
                                                                 .reset_index(drop=True)
                                                                 [['displacement','signal','smoothed']]
                    )

                    df['original'] = 0
                    df.loc[df.timestamp.isin(df_orig.timestamp.unique()), 'original'] = 1


                    df['onset'] = onset
                    df['transient'] = transient
                    df['deformation'] = deformation
                    df['seed'] = seed

                    df_area.append(df)
                    

    df_area = pd.concat(df_area)
            
    filename = f'GA_{area_name[3:]}.parq'
    df_area.to_parquet(dataset_path + filename)
                    

In [ ]:
# Formats a file with the dataset and metadata ready to be used with pytorch 

def format_datasets(dataset_path, data='signal', original=1):

    onset_options = [1, 2, 3]
    transient_options = [1, 2, 3]
    deformation_options = [1, 2]
    num_samples = 2

    metadata = {
        'onset_options': onset_options,
        'transient_options': transient_options,
        'deformation_options': deformation_options,
        'num_samples': num_samples,
        'dataset_path': dataset_path,
    }    

    datasets = []

    for filename in os.listdir(dataset_path):
        if filename.endswith('.parq'):
            df = pd.read_parquet(dataset_path+filename)

            G = fd.NNGraph(df.drop_duplicates(['pid'])[['easting','northing']], radius=15)
            edge_index, edge_weight = dense_to_sparse(torch.tensor(G.W.toarray()).float())
            
            print('file: ' + filename)
            for onset in onset_options:
                for transient in transient_options:
                    for deformation in deformation_options:
                        for seed in range(num_samples):
                            df_sample = df[(df.onset==onset) & (df.transient==transient) &
                                           (df.deformation==deformation) & (df.seed==seed) & (df.original==original)]
                            datasets.append(
                                {'metadata':{'filename':filename, 'onset':onset, 'transient':transient,
                                             'deformation':deformation, 'seed':seed,
                                             'data':data, 'original':original},
                                 'data': df_sample[data].values.reshape((df_sample.pid.nunique(),-1)),
                                 'label': df_sample.label.values.reshape((df_sample.pid.nunique(),-1)),
                                 'pos':df_sample.drop_duplicates(['pid'])[['easting','northing']].values,
                                 'G':G,
                                 'edge_index':edge_index,
                                 'edge_weight':edge_weight,
                                }
                            )

    return datasets, metadata


In [ ]:
datasets, metadata = format_datasets(dataset_path+'Test/')
torch.save(datasets, dataset_path+"Test/dataset.pt")
torch.save(metadata, dataset_path+"Test/dataset_metadata.pt")

#### Phase anomaly (affected pixels)

In [ ]:
basedata_path = root_dir + "/data/interim/TGRS_basedata/"
dataset_path = root_dir + "/data/datasets/Phase_anomaly/"

In [ ]:
# Generates dataframes with anoamlous data
warnings.simplefilter("ignore", DeprecationWarning)

random_seed = 0
np.random.seed(random_seed) # 42 for training, 0 for testing

num_samples = 20
anomalous_nodes = 5
mask_size = 250

selected_area_filenames = []
df_orig_names = []
for filename in os.listdir(basedata_path):
    if 'area' in filename and filename.endswith('.parq'):
        selected_area_filenames.append(filename)
        df_orig_names.append(filename.split('.')[0][:-6])

df_orig_names = list(set(df_orig_names))
df_orig_dict = {name:pd.read_parquet(basedata_path + name + '.parq') for name in df_orig_names}

metadata = {
    'num_samples': num_samples,
    'anomalous_nodes': anomalous_nodes,
    'mask_size': mask_size,
    'basedata_path': basedata_path,
    'dataset_path': dataset_path,
    'selected_area_filenames': selected_area_filenames,
    'df_orig_names': df_orig_names,
}
torch.save(metadata, dataset_path+"data_metadata.pt")

for selected_area_filename in selected_area_filenames:

    area_name = selected_area_filename.split('.')[0]
    
    df_selected = pd.read_parquet(basedata_path + selected_area_filename)

    mask = ( (df_selected.easting < (df_selected.easting.min()+mask_size))
              & (df_selected.northing < (df_selected.northing.min()+mask_size))
    )
    masked_pids = df_selected[mask].pid.unique()
  
    df_orig_name = area_name[:-6]
    df_orig = df_orig_dict[df_orig_name][df_orig_dict[df_orig_name].pid.isin(masked_pids)].copy()

    df_area = []

    for seed in range(num_samples):

        df = df_orig.copy()
        df = df.groupby('pid', as_index=False).apply(resample_df).reset_index().drop('level_0', axis=1)

        node_positions = df.drop_duplicates('pid')[['easting','northing']].values
        anomaly_matrix, label = utils.add_phase_anomaly(node_positions, df.timestamp.nunique(), anomalous_nodes)
        df['anomaly'] = anomaly_matrix.reshape((-1,))
        df['label'] = label.reshape((-1,))
        df['signal'] = df.displacement
        df.loc[df['label'] == 1, 'signal'] = df['anomaly']

        df_stamps = df[df.timestamp.isin(df_orig.timestamp.unique())]
        df[['displacement','signal','smoothed']] =  (df_stamps.groupby('pid', as_index=False)
                                                        .apply(interpolate_signals)
                                                        .reset_index(drop=True)
                                                        [['displacement','signal','smoothed']]
        )

        df['original'] = 0
        df.loc[df.timestamp.isin(df_orig.timestamp.unique()), 'original'] = 1

        df['seed'] = seed

        df_area.append(df)
                    

    df_area = pd.concat(df_area)
            
    filename = f'PA_{area_name[3:]}.parq'
    df_area.to_parquet(dataset_path + filename)
                    

In [ ]:
def get_datasets_PA(dataset_path, data='signal', original=1):

    num_samples =  10

    metadata = {
        'num_samples': num_samples,
        'dataset_path': dataset_path,
    }    

    datasets = []

    for filename in os.listdir(dataset_path):
        if filename.endswith('.parq'):
            df = pd.read_parquet(dataset_path+filename)

            G = fd.NNGraph(df.drop_duplicates(['pid'])[['easting','northing']], radius=15)
            edge_index, edge_weight = dense_to_sparse(torch.tensor(G.W.toarray()).float())
            
            print('file: ' + filename)
            for seed in range(num_samples):
                df_sample = df[(df.seed==seed) & (df.original==original)]
                datasets.append(
                    {'metadata':{'filename':filename, 'seed':seed,
                                    'data':data, 'original':original},
                        'data': df_sample[data].values.reshape((df_sample.pid.nunique(),-1)),
                        'label': df_sample.label.values.reshape((df_sample.pid.nunique(),-1)),
                        'pos':df_sample.drop_duplicates(['pid'])[['easting','northing']].values,
                        'G':G,
                        'edge_index':edge_index,
                        'edge_weight':edge_weight,
                    }
                )

    return datasets, metadata

In [ ]:
datasets, metadata = get_datasets_PA(dataset_path + 'Training/')
torch.save(datasets, dataset_path+"Training/dataset.pt")
torch.save(metadata, dataset_path+"Training/dataset_metadata.pt")

#### Both geological and phase anomaly

In [ ]:
basedata_path = root_dir + "/data/interim/TGRS_basedata/"
dataset_path = root_dir + "/data/datasets/EGMS_anomaly/"

In [ ]:
# Generates dataframes with anoamlous data
warnings.simplefilter("ignore", DeprecationWarning)

random_seed = 42
np.random.seed(random_seed) # 42 for training, 0 for testing

onset_options = [1, 2, 3]
transient_options = [1, 2, 3]
deformation_options = [1, 2]
num_samples = 2
anomaly_radius = 40
mask_size = 250
anomaly_level_factor = 0.75
anomalous_nodes = 5

selected_area_filenames = []
df_orig_names = []
for filename in os.listdir(basedata_path):
    if 'area' in filename and filename.endswith('.parq'):
        selected_area_filenames.append(filename)
        df_orig_names.append(filename.split('.')[0][:-6])

df_orig_names = list(set(df_orig_names))
df_orig_dict = {name:pd.read_parquet(basedata_path + name + '.parq') for name in df_orig_names}

metadata = {
    'seed': random_seed,
    'onset_options': onset_options,
    'transient_options': transient_options,
    'deformation_options': deformation_options,
    'num_samples': num_samples,
    'anomaly_radius': anomaly_radius,
    'mask_size': mask_size,
    'anomaly_level_factor': anomaly_level_factor,
    'anomalous_nodes': anomalous_nodes,
    'basedata_path': basedata_path,
    'dataset_path': dataset_path,
    'selected_area_filenames': selected_area_filenames,
    'df_orig_names': df_orig_names,
}
torch.save(metadata, dataset_path+"data_metadata.pt")

for selected_area_filename in selected_area_filenames:

    area_name = selected_area_filename.split('.')[0]
    
    df_selected = pd.read_parquet(basedata_path + selected_area_filename)

    mask = ( (df_selected.easting < (df_selected.easting.min()+mask_size))
              & (df_selected.northing < (df_selected.northing.min()+mask_size))
    )
    masked_pids = df_selected[mask].pid.unique()
  
    df_orig_name = area_name[:-6]
    df_orig = df_orig_dict[df_orig_name][df_orig_dict[df_orig_name].pid.isin(masked_pids)].copy()

    df_area = []
    for onset in onset_options:
        for transient in transient_options:
            for deformation in deformation_options:
                for seed in range(num_samples):

                    df = df_orig.copy()
                    df = df.groupby('pid', as_index=False).apply(resample_df).reset_index().drop('level_0', axis=1)

                    node_positions = df.drop_duplicates('pid')[['easting','northing']].values
                    anomaly_matrix, label = utils.add_anomaly(node_positions, df.timestamp.nunique(),
                                                              anomaly_radius,
                                                              onset,
                                                              transient,
                                                              deformation)
                    anomaly_level = df_orig.groupby('pid').displacement.agg(np.ptp).quantile(anomaly_level_factor)
                    df['signal'] = df.displacement + anomaly_level*anomaly_matrix.reshape((-1,))
                    df['label'] = label.reshape((-1,))


                    mask = np.where(label.any(axis=1))[0]
                    ####
                    anomaly_matrix, label = utils.add_phase_anomaly_masked(node_positions,
                                                                           df.timestamp.nunique(),
                                                                           anomalous_nodes,
                                                                           mask)
                    df['anomaly'] = anomaly_matrix.reshape((-1,))
                    df['phase_label'] = label.reshape((-1,))
                    df.loc[df['phase_label'] == 1, 'signal'] = df['anomaly']
                    df.loc[df['phase_label'] == 1, 'label'] = 2
                    ####

                    df_stamps = df[df.timestamp.isin(df_orig.timestamp.unique())]
                    df[['displacement','signal','smoothed']] =  (df_stamps.groupby('pid', as_index=False)
                                                                 .apply(interpolate_signals, include_groups=True)
                                                                 .reset_index(drop=True)
                                                                 [['displacement','signal','smoothed']]
                    )

                    df['original'] = 0
                    df.loc[df.timestamp.isin(df_orig.timestamp.unique()), 'original'] = 1


                    df['onset'] = onset
                    df['transient'] = transient
                    df['deformation'] = deformation
                    df['seed'] = seed

                    df_area.append(df)
                    

    df_area = pd.concat(df_area)
            
    filename = f'EA_{area_name[3:]}.parq'
    df_area.to_parquet(dataset_path + filename)
                    

In [ ]:
datasets, metadata = format_datasets(dataset_path+'Test/')
torch.save(datasets, dataset_path+"Test/dataset.pt")
torch.save(metadata, dataset_path+"Test/dataset_metadata.pt")

In [ ]:
px.line(df_area.drop_duplicates(['pid','timestamp']).query('label==2'), x='timestamp', y='signal', animation_frame='pid').show()

In [ ]:
px.scatter(df_area.drop_duplicates('pid'), x='easting', y='northing', color='label', hover_data=['pid'],
           color_continuous_scale=px.colors.sequential.Plasma, title='EGMS Anomaly')

### Visualization

In [ ]:
df = pd.read_parquet(dataset_path + 'Oslo/training/ds_df_Oslo_area1.parq')

In [ ]:
plotting_params = {'edge_color':'darkgray', 'edge_width':1,'vertex_color':'black', 'vertex_size':25}
G = fd.NNGraph(df.drop_duplicates('pid')[['easting','northing']], radius=15, plotting_params=plotting_params)

# can you zoom in in the plot?
fig, ax = plt.subplots(figsize=(10,5))

# G.plot(plot_name='', ax=ax)
G.plot_signal(df.drop_duplicates('pid').displacement.values, ax=ax, colorbar=True, plot_name='', show_edges=True)
#remove box from ax
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False) 
#remove ticks from ax
ax.tick_params(axis='both', which='both', length=0)
#remove labels from ax
ax.set_yticklabels([])
ax.set_xticklabels([])


# #change the colorbar map
cbar = ax.collections[0].colorbar
cbar.set_label('Displacement (mm)', rotation=90, labelpad=10, fontsize=14)
ax.collections[0].set_cmap('coolwarm')

# #remove box around colorbar
cbar.outline.set_visible(False)


#change colorbar tick font size
cbar.ax.tick_params(labelsize=12)

# Add contextily basemap
cx.add_basemap(plt.gca(), crs='EPSG:3035', source=cx.providers.CartoDB.Positron)
# cx.add_basemap(ax, crs='epsg:25833', source=cx.providers.OpenStreetMap.Mapnik)

# plt.savefig('/home/vitorro/Repositories/stae/outputs/figs/EGMS_graph_example_plt.png', dpi=300)
plt.show()

In [ ]:
fig = px.scatter(df[df.pid==df.pid.unique()[0]].query('original==1'), x='timestamp', y='displacement', width=800,
                 labels={'displacement':'Displacement (mm)', 'timestamp':'Date'})

# Make the background transparent
# fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',)
fig.update_xaxes(tickformat='%Y-Q%q', dtick='M3', title_font={'size': 16}, tickfont={'size': 14}) 
fig.update_yaxes(title_font={'size': 18}, tickfont={'size': 14})
fig.show()

In [ ]:
df_sample = df.query('onset==1 and transient==1 and deformation==1 and seed==0')
X = df_sample.signal.values.reshape((df_sample.pid.nunique(),-1))
px.line(X[3,:],width=1000).show()
df

In [ ]:
dataset_path = root_dir + "/data/datasets/"
dataset_name = 'EGMS_anomaly/'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')

In [ ]:
label = datasets[0]['label'].max(axis=1)

In [ ]:
datasets[0].keys()

In [ ]:
px.scatter(x=datasets[0]['pos'][:,0], y=datasets[0]['pos'][:,1], color=label, color_continuous_scale=px.colors.sequential.Plasma)

In [ ]:
datasets[20]['metadata']

In [ ]:
dataset_path = root_dir + "/data/datasets/"
dataset_name = 'EGMS_anomaly/'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
dataset = datasets[20]

label = dataset['label'].max(axis=1)

# Create figure and plot
fig, ax = plt.subplots(figsize=(5,5))

# Create custom colormap for 3 categories
colors = [(0.5, 0.5, 0.8), (0.05, 0.25, 0.25), (0.55, 0.55, 0.05)]  # blue, orange, green
categories = ['Normal', 'Geological', 'Phase']
scatter_plots = []

# Plot each category separately to create legend entries
for i, cat in enumerate(np.unique(label)):
    mask = label == cat
    scatter = ax.scatter(dataset['pos'][mask,0], dataset['pos'][mask,1], 
                        c=[colors[int(cat)]], label=categories[int(cat)], 
                        s=25, alpha=0.7)

# Remove box and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(axis='both', which='both', length=0)
ax.set_yticklabels([])
ax.set_xticklabels([])

# Add legend
ax.legend(title='Label', frameon=True, loc='lower left')

# Add contextily basemap
cx.add_basemap(plt.gca(), crs='EPSG:3035', source=cx.providers.CartoDB.Positron)

plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5,5), tight_layout=True)

# Get indices where label is 1 and 2
idx1 = np.where(label == 1)[0] 
idx2 = np.where(label == 2)[0]

if len(idx1) > 0:
    X1 = dataset['data'][idx1[0],:]
    ax1.plot(X1, label='Geological anomaly')
    ax1.legend(frameon=False, fontsize=12, loc='lower right')
    ax1.set_xticks([])
    # ax1.grid(True)

if len(idx2) > 0:    
    X2 = dataset['data'][idx2[0],:]
    ax2.plot(X2, label='Phase anomaly')
    ax2.legend(frameon=False, fontsize=12)
    # ax2.grid(True)

plt.show()

In [ ]:
dataset_path = root_dir + "/data/datasets/"
dataset_name = 'EGMS_anomaly/'
datasets = torch.load(dataset_path + f'{dataset_name}/Test/dataset.pt')
dataset = datasets[20]

label = dataset['label'].max(axis=1)

# Create figure with 1x3 subplot layout 
fig = plt.figure(figsize=(10,5))

gs = gridspec.GridSpec(2, 2)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])

# Scatter plot
colors = [(0.5, 0.5, 0.8), (0.05, 0.25, 0.25), (0.55, 0.55, 0.05)]
categories = ['Normal', 'Geological', 'Phase']

for i, cat in enumerate(np.unique(label)):
    mask = label == cat
    ax1.scatter(dataset['pos'][mask,0], dataset['pos'][mask,1], 
                c=[colors[int(cat)]], label=categories[int(cat)], 
                s=25, alpha=0.7)

# Clean up scatter plot
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.tick_params(axis='both', which='both', length=0)
ax1.set_yticklabels([])
ax1.set_xticklabels([])
ax1.legend(title='', frameon=True, loc='upper right', fontsize=12)

# Add basemap
cx.add_basemap(ax1, crs='EPSG:3035', source=cx.providers.CartoDB.Positron)

# Time series plots
idx1 = np.where(label == 1)[0] 
idx2 = np.where(label == 2)[0]


X1 = dataset['data'][idx1[0],:]
ax2.plot(X1, color=colors[1])
ax2.set_title('Geological anomaly', fontsize=12, loc='left')
ax2.set_xticks([])

X2 = dataset['data'][idx2[0],:]
ax3.plot(X2, color=colors[2])
ax3.set_title('Phase anomaly', fontsize=14, loc='left')
ax3.tick_params(axis='x', direction='in')
ax3.set_xticks([])
ax3.set_xlabel('Time', fontsize=14)

# Configure time series axes
for ax in [ax2, ax3]:
    # Hide default y-axis ticks
    ax.tick_params(axis='y', left=False)
    ax.set_yticklabels([])

ax2.set_xlim([0,325])
ax2.annotate('0', xy=(325, 0), xycoords='data', ha='right', fontsize=12)
ax2.annotate('15', xy=(325, 15), xycoords='data', ha='right', fontsize=12)
ax2.annotate('30', xy=(325, 30), xycoords='data', ha='right', fontsize=12)

ax3.set_xlim([0,325])
ax3.annotate('0', xy=(325, -5), xycoords='data', ha='right', fontsize=12)
ax3.annotate('-40', xy=(325, -40), xycoords='data', ha='right', fontsize=12)
ax3.annotate('-80', xy=(325, -80), xycoords='data', ha='right', fontsize=12)

fig.text(0.99, 0.5, 'Displacement (mm)', va='center', rotation='vertical', fontsize=14)

plt.tight_layout(pad=1.0)

plt.savefig('/home/vitorro/Repositories/stae/outputs/figs/EGMS_anomaly_example_plt.png', dpi=300,bbox_inches='tight')

plt.show()


----------------------------

In [ ]:
df_Oslo

In [ ]:
 df_Oslo = pd.read_parquet('/home/vitorro/Repositories/stae/data/interim/df_Oslo.parq')

In [ ]:
df_Oslo[['easting','northing']].describe()

In [ ]:
plt.scatter(df_Oslo.drop_duplicates('pid')[['easting','northing']].values[:,0] - df_Oslo.drop_duplicates('pid')[['easting','northing']].values[:,0].min(),
            df_Oslo.drop_duplicates('pid')[['easting','northing']].values[:,1] - df_Oslo.drop_duplicates('pid')[['easting','northing']].values[:,1].min(),
            s=1, alpha=0.5, c='black')
cx.add_basemap(plt.gca(), crs='EPSG:3035', source=cx.providers.CartoDB.Positron)
plt.title('Oslo area')
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.show()

In [ ]:
datasets = []

for east in range(int(df_Oslo.easting.min()), int(df_Oslo.easting.max()), 200):
    for north in range(int(df_Oslo.northing.min()), int(df_Oslo.northing.max()), 200):
        mask_size = 250
        mask = ( (df_Oslo.easting>east) & (df_Oslo.easting < (east+mask_size)) &
                 (df_Oslo.northing>north) & (df_Oslo.northing < (north+mask_size))
        )
        masked_pids = df_Oslo[mask].pid.unique()

        # Skip if number of pids is greater than 50
        if len(masked_pids) < 50:
            continue

        df_mask = df_Oslo[df_Oslo.pid.isin(masked_pids)].copy()

        while df_mask.drop_duplicates('pid')[['easting','northing']].duplicated(['easting','northing']).values.any():
            fix_pid = df_mask.drop_duplicates('pid')[df_mask.drop_duplicates('pid')[['easting','northing']].duplicated(['easting','northing'], keep='first').values].pid.unique()
            for pid in fix_pid:
                df_mask.loc[df_mask.pid==pid, 'easting'] += np.random.uniform(-0.5, 0.5)

        G = fd.NNGraph(df_mask.drop_duplicates('pid')[['easting','northing']], radius=15)
        edge_index, edge_weight = dense_to_sparse(torch.tensor(G.W.toarray()).float())

        datasets.append(
            {'metadata':{'easting':east, 'northing':north, 'mask_size':mask_size},
             'data': df_mask.displacement.values.reshape((df_mask.pid.nunique(),-1)),
             'pos':df_mask.drop_duplicates(['pid'])[['easting','northing']].values,
             'coords':df_mask.drop_duplicates(['pid'])[['latitude','longitude']].values,
             'pid':df_mask.drop_duplicates(['pid']).pid.values,
             'G':G,
             'edge_index':edge_index,
             'edge_weight':edge_weight,
            }
        )


In [ ]:
torch.save(datasets, '/home/vitorro/Repositories/stae/data/datasets/Real_data/dataset.pt')

In [ ]:
[d['G'].N for d in datasets]

In [ ]:
df_mask[['easting','northing']].duplicated(['easting','northing'])

In [ ]:
np.where(np.diag(G.W.toarray())>0)